In [1]:
# install packages
%pip install pandas
%pip install plotly


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import packages
import pandas
import plotly.express
import math

In [3]:
def calculate_clo(m: float, icl: float) -> float:
    """
    Calculates the clo for a given metabolic rate and clothing insulation using ASHRAE 55:2023.
    
    Args:
        m (float): metabolic rate in met unit
        icl (float): clothing insulation in clo unit
    
    Return:
        float: clo value to use in the PMV calculator
    """
    if m < 1.2:
        clo = icl

    else:
        clo = icl * (0.6 + 0.4 / m)

    return clo

In [5]:
def calculate_pmv(ta: float, tr: float, vav: float, rh: float, met: float, clo: float, wme: float) -> dict:
     """
     Calculate the PMV for given inputs using ASHRAE 55:2023.

     Args:
          ta (float): air temperature in C
          tr (float): mean radiant temperature in C
          vav(float): average air speed in m/s
          vel_act (float): activity-generated air speed m/s
          rh (float): relative humidity in %
          met (float): metabolic rate in met unit
          clo (float): clothing factor in clo unit (calculated using calculate_clo method)
          wme (float): external work in met

     Return:
          dict: {
               pmv: Predicted Mean Vote (PMV), 
               ppd: Predicted Precentage of Dissatisfied (PPD)
               }
     """
     if met > 1:
          pa = rh * 10 * math.exp(16.6536 - 4030.183 / (ta + 235))
          icl = 0.155 * clo # thermal insulation of the clothing in m2K/W
          m = met * 58.15 # metabolic rate in W/m2
          w = wme * 58.15 # extenal work in W/m2
          mw = m - w # internal heat production in the humen body
          vag = 0.3 * (met - 1) # activity generated metabolic rate
          vel = vav + vag

          if icl <= 0.078:
               fcl = 1 + (1.29 * icl)
          else:
               fcl = 1.05 + (0.645 * icl)

          # heat transfer coefficient by forced convection
          hcf = 12.1 * math.sqrt(vel)
          taa = ta + 273
          tra = tr + 273
          tcla = taa + (35.5 - ta) / (3.5 * icl + 0.1)
          p1 = icl * fcl

     else:
          pmv = 0.0
          ppd = 0.0
          print("ASHRAE 55:2023 calculation does not available for metabolic rate less than 1")
     
     return {
          "pmv": pmv,
          "ppd": ppd
          }